# Capstone Term 2 - Project

* Marcos Bittencourt
---
* Contributors
    * Todd McCullough [Git](https://github.com/tamccullough)
    * Savya Sharma [Git](https://github.com/SavyaSharma)
    * Marko Topitch [Git](https://github.com/TopMarko)
---

### Load the Model and Data

##### Import the Needed Modules

In [11]:
import pandas as pd
import numpy as np
import heapq
import pickle

##### Load the production ready model

In [12]:
filename = 'recipes_recommender_model.sav'
rr_model = pickle.load(open(filename, 'rb'))

##### Import Data

In [13]:
recipes_df = pd.read_csv('datasets/rr-recipes.csv')
users_df = pd.read_csv('datasets/rr-users.csv')
ratings_df = pd.read_csv('datasets/rr-ratings.csv')

##### Test Prediction

In [14]:
i = 1
for i in range(150):
    prediction = rr_model.predict(i,167)
    print(round(prediction.est,2), end=', ')
    i = i + 1

4.46, 4.38, 4.65, 4.65, 4.65, 5, 4.65, 4.65, 4.65, 4.65, 4.46, 4.26, 4.65, 4.65, 4.65, 4.46, 4.65, 4.65, 5, 4.46, 4.65, 4.65, 4.65, 4.65, 4.65, 4.65, 4.65, 4.65, 4.17, 4.65, 5, 4.65, 4.65, 4.65, 4.65, 4.65, 4.65, 4.46, 4.65, 4.65, 4.65, 4.46, 4.65, 4.65, 4.65, 4.65, 4.65, 4.65, 4.46, 4.65, 4.46, 4.65, 4.65, 4.46, 3.76, 4.76, 5, 3.01, 5, 5, 5, 4.76, 2.76, 5, 4.46, 5, 5, 5, 4.76, 5, 1.76, 2.76, 5, 5, 4.65, 4.47, 5, 5, 4.76, 4.76, 2.76, 5, 3.76, 1.76, 1.76, 1.76, 3.76, 4.76, 3.76, 5, 4.46, 2.76, 4.46, 2.76, 1.76, 4.46, 5, 5, 5, 4.46, 4.76, 4.87, 4.65, 4.76, 5, 4.46, 5, 5, 4.76, 5, 5, 4.46, 5, 2.76, 4.46, 5, 5, 4.76, 4.46, 5, 4.46, 3.76, 4.76, 5, 5, 5, 4.8, 4.45, 5, 4.46, 5, 5, 4.76, 4.76, 5, 2.76, 4.46, 4.46, 2.76, 5, 4.76, 4.65, 5, 5, 3.76, 5, 4.76, 3.76, 4.46, 4.46, 

### Recommendations

In [15]:
def get_r(user_id):
    # Select which system to use. Due to memory constraints, item based is the only viable option
    recommender_system = rr_model

    # N will represent how many items to recommend
    N = 200

    # The setting to a set and back to list is a failsafe.
    rated_items = list(set(ratings_df.loc[ratings_df['user'] == user_id]['item'].tolist()))

    # Self explanitory name
    all_item_ids = list(set(ratings_df['item'].tolist()))

    # New_items just represents all the items not rated by the user
    new_items = [x for x in all_item_ids if x not in rated_items]

    # Estimate ratings for all unrated items
    predicted_ratings = {}
    for item_id in new_items:
        predicted_ratings[item_id] = recommender_system.predict(user_id, item_id).est
        pass

    # Get the item_ids for the top ratings
    recommended_ids = heapq.nlargest(N, predicted_ratings, key=predicted_ratings.get)
    recommended_ids = sorted(recommended_ids)

    # predicted_ratings
    recommended_df = recipes_df.loc[recipes_df['recipe_id'].isin(recommended_ids)].copy()
    recommended_df.set_index('recipe_id', inplace=True)
    recommended_df.insert(1, 'pred_rating', np.zeros(len(recommended_ids)))
    # recommended_df = recipes_df.copy()
    for idx,item_id in enumerate(recommended_ids):
        recommended_df.iloc[idx, recommended_df.columns.get_loc('pred_rating')] =predicted_ratings[item_id]
        pass

    return recommended_df.head(N).sort_values('pred_rating', ascending=False)

### Get a Recommendation Based on Ingredients

The final code that will be impletented in a cleaner fashion through the browser interface.

In [16]:
# ask the user for input
# get their ID number
user_id = int(input('Enter user id: '))

# get them to list some ingredients, currently it breaks if the second or next ingredient is not there
ingredient_list = input('Enter the ingredients separated by commas that you have on hand: ')

# split the input up into an array for the loop
items = np.array(ingredient_list.split(','))

# get the lowest rating
rating = int(input('Enter the lowest rating you\'ll accept: '))

# get their user name
user_name = users_df.loc[users_df['user_id'] == user_id]

# print some details
print('\nuser: ',user_name.iloc[0,1])
print(ingredient_list)
print('\nHere are your recommendations.')
test = get_r(user_id)
for item in items:
    test = test[test['ingredients'].str.contains(item)]
test = test[test['pred_rating'] >= rating]
test

Enter user id: 35
Enter the ingredients separated by commas that you have on hand: cheese
Enter the lowest rating you'll accept: 5

user:  35
cheese

Here are your recommendations.


,Unnamed: 0,pred_rating,title,servings,cals_per_serving,prep_time,cook_time,ready_time,nutrition,ingredients,url
recipe_id,,,,,,,,,,,
505,504,5.0,Butter Schnitzel,12,351.0,25 Minutes,30 Minutes,55 m,Per Serving: 351 calories;23.2 g fat;15.7g car...,"12 boneless pork loin chops, 3/4 inch thick;2 ...",https://www.allrecipes.com/recipe/109451/butte...
967,966,5.0,Almond Melon Tart,8,437.0,30 Minutes,30 Minutes,3 h,Per Serving: 437 calories;25.6 g fat;47.2g car...,1/2 (11 ounce) package pie crust mix;1 cup sou...,https://www.allrecipes.com/recipe/12370/almond...
891,890,5.0,Chile Rellenos Pie,6,289.0,15 Minutes,45 Minutes,1 h,Per Serving: 289 calories;19.2 g fat;18.2g car...,4 eggs;1/2 cup milk;1/4 cup chopped fresh cila...,https://www.allrecipes.com/recipe/12115/chile-...
869,868,5.0,Classic Alfredo Sauce,2,714.0,None,None,None,Per Serving: 714 calories;72 g fat;5g carbohyd...,3 tablespoons butter;8 fluid ounces heavy whip...,https://www.allrecipes.com/recipe/12065/classi...
851,850,5.0,Alfredo Light,8,292.0,20 Minutes,20 Minutes,40 m,Per Serving: 292 calories;4.1 g fat;50.5g carb...,"1 onion, chopped;1 clove garlic, minced;2 teas...",https://www.allrecipes.com/recipe/11915/alfred...
828,827,5.0,Baked Ziti I,10,578.0,20 Minutes,35 Minutes,55 m,Per Serving: 578 calories;25.3 g fat;58.4g car...,"1 pound dry ziti pasta;1 onion, chopped;1 poun...",https://www.allrecipes.com/recipe/11758/baked-...
821,820,5.0,American Lasagna,8,664.0,30 Minutes,1 Hour 15 Minutes,1 h 55 m,Per Serving: 664 calories;29.5 g fat;48.3g car...,"1 1/2 pounds lean ground beef;1 onion, chopped...",https://www.allrecipes.com/recipe/11729/americ...
820,819,5.0,Cheese Ravioli with Fresh Tomato and Artichoke...,6,355.0,20 Minutes,5 Minutes,25 m,Per Serving: 355 calories;15.5 g fat;42g carbo...,2 (9 ounce) packages fresh cheese ravioli;1 ta...,https://www.allrecipes.com/recipe/11723/cheese...
815,814,5.0,Alla Checca,4,610.0,20 Minutes,None,2 h 20 m,Per Serving: 610 calories;30.7 g fat;69.4g car...,"5 tomatoes, seeded and diced;4 cloves garlic, ...",https://www.allrecipes.com/recipe/11692/alla-c...
